In [9]:
import numpy as np
import cv2 

In [10]:
def TV_noise_2d(u0, N=300, weight=60, eps = 2e-4):
    """implements ROF model according to primal-dual algorithm from
    "Algoithm for total variation minimization and applications", Chambolle,
    2004. 
    """
    px = np.zeros_like(u0)
    py = np.zeros_like(u0)
    gx = np.zeros_like(u0)
    gy = np.zeros_like(u0)
    d = np.zeros_like(u0)
    i = 0
    while i < N:
        d = -px - py
        d[1:] += px[:-1]
        d[:, 1:] += py[:, :-1] 
        # div scheme
        out = u0 + d
        E = (d**2).sum()
        gx[:-1] = np.diff(out, axis=0) 
        gy[:, :-1] = np.diff(out, axis=1) 
        norm = np.sqrt(gx**2 + gy**2)
        E += weight * norm.sum()
        norm *= 0.5 / weight
        norm += 1
        px -= 0.25*gx
        px /= norm
        py -= 0.25*gy
        py /= norm
        E /= float(u0.size)
        if i == 0:
            E_init = E
            E_previous = E
        else:
            if np.abs(E_previous - E) < eps * E_init:
                break
            else:
                E_previous = E
        i += 1
    return out

In [13]:
if __name__ == "__main__":
    fileName = "input.png"
    N = 500
    eps = 2.e-4
    weight = 40

    u = cv2.imread("input.png", cv2.IMREAD_GRAYSCALE)
    print(u)
    u = u.astype(np.float32)#/ 255.0

    result = TV_noise_2d(u, N, weight=weight, eps=eps)
    #result *= 255
    result = result.astype(np.uint8)
    cv2.imwrite("output_chambolle.png", result)

[[162 166 144 ... 167 194 108]
 [192 143 189 ... 161 161 141]
 [170 210 214 ... 154 146 141]
 ...
 [  0   4  74 ...  67 108 128]
 [ 48  97  45 ... 130 144 120]
 [ 63   0  63 ... 114  80  83]]
